In [3]:
from qiskit import QuantumCircuit, execute
import qiskit.quantum_info as qi
#the simulator
from qiskit.providers.aer import AerSimulator
# Noise model
from qiskit.test.mock import FakeVigo
from numpy import pi

#qiskit experiments is replacing qiskit ignis. Docs can be found here:
#https://qiskit.org/documentation/experiments/apidocs/index.html
from qiskit_experiments.library.tomography import ProcessTomography, ProcessTomographyAnalysis
from qiskit_experiments.library.tomography.fitters import linear_inversion, scipy_gaussian_lstsq

#Is there a library for SICPOVM basis?
from qiskit_experiments.library.tomography.basis import PauliMeasurementBasis, PauliPreparationBasis

# Construct noisy simulator
backend = AerSimulator.from_backend(FakeVigo())

1. Prepare the process by specifying a circuit

In [4]:
qc = QuantumCircuit(1)
qc.ry(pi/2, 0);

2. The tomography experiment sets up the circuits to measure in the chosen basis

In [17]:
#circuit (or appendable object, measurement basis, qubits to measure (all if None), and preparation basis)
exp = ProcessTomography(qc)
experiment_data = exp.run(backend, fitter = "linear_inversion")

In [18]:
#acquire the data
results = experiment_data.analysis_results()
choi_matrix = results[0].value
kraus_rep = qi.Kraus(choi_matrix)

In [19]:
print(experiment_data.jobs()[0].result().get_counts())

[{'0': 495, '1': 529}, {'1': 62, '0': 962}, {'1': 496, '0': 528}, {'1': 507, '0': 517}, {'0': 91, '1': 933}, {'0': 521, '1': 503}, {'0': 73, '1': 951}, {'0': 514, '1': 510}, {'1': 535, '0': 489}, {'1': 477, '0': 547}, {'1': 545, '0': 479}, {'1': 90, '0': 934}]


In [6]:
for (i,op) in enumerate(kraus_rep.data):
    print("Kraus operator " + str(i) + ":\n")
    print(op)
    print("\n")

Kraus operator 0:

[[ 0.66102454-2.99258647e-17j -0.68884136-2.07222535e-02j]
 [ 0.68036113-1.74846107e-02j  0.65297883+3.15234914e-02j]]


Kraus operator 1:

[[-0.07678613-5.09359511e-16j -0.01553209-7.36176057e-02j]
 [-0.06540309-5.42950125e-03j  0.13007581-6.34798216e-02j]]


Kraus operator 2:

[[ 8.78525773e-11-0.17799218j  3.21189909e-02-0.03562504j]
 [ 4.38259930e-02+0.17240365j -6.38655752e-03-0.03953072j]]


Kraus operator 3:

[[-0.10861564+1.82573960e-14j -0.10838458+1.98844318e-01j]
 [-0.0607013 +1.04112767e-01j  0.06268127+1.09377444e-01j]]




In [7]:
exp.circuits()[0].draw()

┌─────────────┐ ░ ┌─────────┐ ░ ┌────────────┐ ░ ┌─┐
  q: ─|0>─┤ PauliPrepZp ├─░─┤ Ry(π/2) ├─░─┤ PauliMeasZ ├─░─┤M├
          └─────────────┘ ░ └─────────┘ ░ └────────────┘ ░ └╥┘
c: 1/═══════════════════════════════════════════════════════╩═
                                                            0

In [8]:
print(choi_matrix)

Choi([[ 0.48632813+0.j        ,  0.43066406+0.01464844j,
       -0.43603516+0.02392578j,  0.421875  -0.01269531j],
      [ 0.43066406-0.01464844j,  0.51367188+0.j        ,
       -0.44433594+0.02929688j,  0.43603516-0.02392578j],
      [-0.43603516-0.02392578j, -0.44433594-0.02929688j,
        0.53417969+0.j        , -0.43164062+0.0234375j ],
      [ 0.421875  +0.01269531j,  0.43603516+0.02392578j,
       -0.43164062-0.0234375j ,  0.46582031+0.j        ]],
     input_dims=(2,), output_dims=(2,))
